## Setup

In [ ]:
%matplotlib qt
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import os
import json

In [ ]:
Path('Results').mkdir(exist_ok=True)
os.chdir('Results')

In [ ]:
colors = [
    '#1f77b4',
    '#ff7f0e',
    '#2ca02c',
    '#d62728',
    '#9467bd',
    '#8c564b',
    '#e377c2',
    '#7f7f7f',
    '#bcbd22',
    '#17becf',
    '#1f77b4',
    '#ff7f0e'
]

In [ ]:
top_folder = os.path.abspath(os.path.join('..', '..'))
gan_folder = os.path.join(top_folder, 'GAN')
dcgan_folder = os.path.join(top_folder, 'DCGAN')
cgan_folder = os.path.join(top_folder, 'CGAN')
wgan_folder = os.path.join(top_folder, 'WGAN')
wgan_gp_folder = os.path.join(top_folder, 'WGAN-GP')
stylegan_folder = os.path.join(top_folder, 'StyleGAN')

In [ ]:
def get_logs(parent, experiments):
    cs = dict()
    fcd = dict()
    for folder, name in experiments.items():
        with open(os.path.join(parent, folder, 'log.json')) as log_file:
            log = json.load(log_file)
            cs[name] = log['classifier_score']
            fcd[name] = log['frechet_distance']
    return cs, fcd

In [ ]:
def plot_metrics(metrics, xlabel='Epoch', figsize=(12, 4), **kwargs):
    cols = len(metrics)
    axes = list()
    fig = plt.figure(figsize=figsize)
    kwargs.setdefault('linewidth', 1)
    for i, (ylabel, data) in enumerate(metrics.items()):
        axes.append(plt.subplot(1, cols, i + 1))
        for name, values in data.items():
            plt.plot(values, label=name, **kwargs)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.grid()
    plt.tight_layout(w_pad=4)
    return fig, axes

## 1 GAN

### 1.1 MNIST

In [ ]:
experiments = {
    'BS4_LDIM256_SGD' : 'BS=4  LDIM=256 OPT=SGD',
    'BS8_LDIM10_SGD'  : 'BS=8  LDIM=10 OPT=SGD',
    'BS32_LDIM128_SGD': 'BS=32 LDIM=128 OPT=SGD',
    'BS64_LDIM10_SGD' : 'BS=64 LDIM=10 OPT=SGD',
    'BS8_LDIM10_Adam' : 'BS=8  LDIM=10 OPT=Adam'
}
parent = os.path.join(gan_folder, 'MNIST')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd})
[axes.legend(loc='best', framealpha=1, fontsize='small') for axes in [ax1, ax2]]
fig.savefig('GAN_MNIST.pdf')

## 2 DCGAN

### 2.1 MNIST

Plotting all results

In [ ]:
experiments = {
    'BS8_LDIM100_TRPCONV'  : 'BS=8   LDIM=100 UP=TrpConv',
    'BS16_LDIM100_TRPCONV' : 'BS=16  LDIM=100 UP=TrpConv',
    'BS32_LDIM100_TRPCONV' : 'BS=32  LDIM=100 UP=TrpConv',
    'BS64_LDIM100_TRPCONV' : 'BS=64  LDIM=100 UP=TrpConv',
    'BS128_LDIM100_TRPCONV': 'BS=128 LDIM=100 UP=TrpConv',
    
    'BS16_LDIM2_BILINEAR'  : 'BS=16 LDIM=2   UP=Bilinear',
    'BS16_LDIM50_BILINEAR' : 'BS=16 LDIM=50  UP=Bilinear',
    'BS16_LDIM100_BILINEAR': 'BS=16 LDIM=100 UP=Bilinear',
    'BS16_LDIM128_BILINEAR': 'BS=16 LDIM=128 UP=Bilinear',
    
    'BS32_LDIM6_BILINEAR'  : 'BS=32 LDIM=6   UP=Bilinear',
    'BS32_LDIM12_BILINEAR' : 'BS=32 LDIM=12  UP=Bilinear',
    'BS32_LDIM50_BILINEAR' : 'BS=32 LDIM=50  UP=Bilinear',
    'BS32_LDIM100_BILINEAR': 'BS=32 LDIM=100 UP=Bilinear',
    'BS32_LDIM128_BILINEAR': 'BS=32 LDIM=128 UP=Bilinear'
}
parent = os.path.join(dcgan_folder, 'MNIST')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 6))
ax1.get_lines()[-1].set_color(colors[5])
ax2.get_lines()[-1].set_color(colors[5])
[axes.legend(loc='best', framealpha=1, fontsize='small') for axes in [ax1, ax2]]
fig.savefig('DCGAN_MNIST.pdf')

Highlighting differences from bilinear upsample and transposed convolution

In [ ]:
lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
[line.set_color(colors[0]) for line in [*lines1[0:5], *lines2[0:5]]]
[line.set_color(colors[1]) for line in [*lines1[5:], *lines2[5:]]]
[axes.legend([lines1[0], lines1[-1]], ['Transpose Convolution', 'Bilinear Upsample']) for axes in [ax1, ax2]]
fig.set_size_inches((12, 4))
plt.tight_layout(h_pad=2, w_pad=4)
fig.savefig('DCGAN_MNIST_Upsampling.pdf')

### 2.2 Fashion MNIST

In [ ]:
experiments = {
    'BS16_LDIM16_TRPCONV': 'BS=16 LDIM=16 UP=TrpConv',
    'BS16_LDIM32_TRPCONV': 'BS=16 LDIM=32 UP=TrpConv',
    'BS32_LDIM16_TRPCONV': 'BS=32 LDIM=16 UP=TrpConv',
    'BS32_LDIM32_TRPCONV': 'BS=32 LDIM=32 UP=TrpConv',
    
    'BS8_LDIM32_NEAREST' : 'BS=8  LDIM=32 UP=Nearest',
    'BS16_LDIM32_NEAREST': 'BS=16 LDIM=32 UP=Nearest',
    'BS32_LDIM32_NEAREST': 'BS=32 LDIM=32 UP=Nearest',
    
    'BS8_LDIM32_BILINEAR' : 'BS=8  LDIM=32 UP=Bilinear',
    'BS16_LDIM16_BILINEAR': 'BS=16 LDIM=16 UP=Bilinear',
    'BS16_LDIM32_BILINEAR': 'BS=16 LDIM=32 UP=Bilinear',
    'BS32_LDIM16_BILINEAR': 'BS=32 LDIM=16 UP=Bilinear',
    'BS32_LDIM32_BILINEAR': 'BS=32 LDIM=32 UP=Bilinear',
    'BS64_LDIM32_BILINEAR': 'BS=64 LDIM=32 UP=Bilinear'
}
parent = os.path.join(dcgan_folder, 'Fashion')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd})
[axes.legend(loc='best', framealpha=1, fontsize='x-small') for axes in [ax1, ax2]]
ax2.set_ylim(0, 12)
fig.savefig('DCGAN_Fashion.pdf')

In [ ]:
lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
[line.set_color(colors[0]) for line in [*lines1[0:4], *lines2[0:4]]]
[line.set_color(colors[2]) for line in [*lines1[4:7], *lines2[4:7]]]
[line.set_color(colors[1]) for line in [*lines1[7:], *lines2[7:]]]
[
    axes.legend(
        [lines1[0], lines1[7], lines1[4]],
        ['Transpose Convolution', 'Bilinear Upsample', 'Nearest Neighbour']
    ) for axes in [ax1, ax2]
]
fig.set_size_inches((12, 4))
fig.savefig('DCGAN_Fashion_Upsampling.pdf')

### 2.3 CIFAR

In [ ]:
experiments = {
    'TRPCONV'        : 'UP=TrpConv',
    'BILINEAR'       : 'UP=Bilinear',
    'TRPCONV_SMOOTH' : 'UP=TrpConv SMOOTH=0.9',
    'BILINEAR_SMOOTH': 'UP=Bilinear SMOOTH=0.9'
}
parent = os.path.join(dcgan_folder, 'CIFAR')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 4))
[axes.legend(loc='best', framealpha=1, fontsize='small') for axes in [ax1, ax2]]
fig.savefig('DCGAN_CIFAR.pdf')

## 3 CGAN

### 3.1 MNIST

In [ ]:
experiments = {
    'BS16': 'BS=16',
    'BS32': 'BS=32',
    'BS16_SMOOTH': 'BS=16 SMOOTH=0.9',
    'BS32_SMOOTH': 'BS=32 SMOOTH=0.9 '
}
parent = os.path.join(cgan_folder, 'MNIST')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 4))
[axes.legend(loc='best', framealpha=1, fontsize='small') for axes in [ax1, ax2]]
fig.savefig('CGAN_MNIST.pdf')

### 3.2 Fashion MNIST

In [ ]:
experiments = {
    'BS16': 'BS=16',
    'BS32': 'BS=32',
    'BS16_SMOOTH': 'BS=16 SMOOTH=0.9',
    'BS32_SMOOTH': 'BS=32 SMOOTH=0.9 '
}
parent = os.path.join(cgan_folder, 'Fashion')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 4))
[axes.legend(loc='best', framealpha=1, fontsize='small') for axes in [ax1, ax2]]
fig.savefig('CGAN_Fashion.pdf')

### 3.3 CIFAR

In [ ]:
experiments = {
    'beta1[0.0]_TRPCONV' : r'$\beta_1$=0.0 UP=TrpConv',
    'beta1[0.5]_TRPCONV' : r'$\beta_1$=0.5 UP=TrpConv',
    'beta1[0.0]_BILINEAR': r'$\beta_1$=0.0 UP=Bilinear',
    'beta1[0.5]_BILINEAR': r'$\beta_1$=0.5 UP=Bilinear',
    'beta1[0.0]_NEAREST' : r'$\beta_1$=0.0 UP=Nearest',
    'beta1[0.5]_NEAREST' : r'$\beta_1$=0.5 UP=Nearest',
    
    'beta1[0.0]_TRPCONV_SMOOTH' : r'$\beta_1$=0.0 UP=TrpConv SMOOTH=0.9',
    'beta1[0.5]_TRPCONV_SMOOTH' : r'$\beta_1$=0.5 UP=TrpConv SMOOTH=0.9',
    'beta1[0.0]_BILINEAR_SMOOTH': r'$\beta_1$=0.0 UP=Bilinear SMOOTH=0.9',
    'beta1[0.5]_BILINEAR_SMOOTH': r'$\beta_1$=0.5 UP=Bilinear SMOOTH=0.9',
    'beta1[0.0]_NEAREST_SMOOTH' : r'$\beta_1$=0.0 UP=Nearest  SMOOTH=0.9',
    'beta1[0.5]_NEAREST_SMOOTH' : r'$\beta_1$=0.5 UP=Nearest  SMOOTH=0.9'
}
parent = os.path.join(cgan_folder, 'CIFAR')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 6))
[axes.legend(loc='best', framealpha=1, fontsize='x-small', ncol=2) for axes in [ax1, ax2]]
fig.savefig('CGAN_CIFAR.pdf')

In [ ]:
lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
ids = {
    'TrpConv' : [0, 1, 6, 7],
    'Bilinear' : [2, 3, 8, 9],
    'Nearest': [4, 5, 10, 11]
}
[(lines1[i].set_color(colors[0]), lines2[i].set_color(colors[0])) for i in ids['TrpConv']]
[(lines1[i].set_color(colors[1]), lines2[i].set_color(colors[1])) for i in ids['Bilinear']]
[(lines1[i].set_color(colors[2]), lines2[i].set_color(colors[2])) for i in ids['Nearest']]
[
    axes.legend(
        [lines1[0], lines1[2], lines1[4]],
        ['Transpose Convolution', 'Bilinear Upsample', 'Nearest Neighbour']
    ) for axes in [ax1, ax2]
]
fig.set_size_inches((12, 4))
fig.savefig('CGAN_CIFAR_Upsampling.pdf')

In [ ]:
lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
ids = {
    'beta1=0.0' : [*range(0, 12, 2)],
    'beta1=0.5' : [*range(1, 12, 2)],
}
[(lines1[i].set_color(colors[0]), lines2[i].set_color(colors[0])) for i in ids['beta1=0.0']]
[(lines1[i].set_color(colors[1]), lines2[i].set_color(colors[1])) for i in ids['beta1=0.5']]
[
    axes.legend(
        [lines1[0], lines1[1]],
        [r'$\beta_1$=0.0', r'$\beta_1$=0.5']
    ) for axes in [ax1, ax2]
]
fig.set_size_inches((12, 4))
fig.savefig('CGAN_CIFAR_beta1.pdf')

In [ ]:
lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
[line.set_color(colors[0]) for line in [*lines1[0:6], *lines2[0:6]]]
[line.set_color(colors[1]) for line in [*lines1[6:], *lines2[6:]]]
[
    axes.legend(
        [lines1[0], lines1[7]],
        ['No Smoothing', 'Smoothing = 0.9']
    ) for axes in [ax1, ax2]
]
fig.set_size_inches((12, 4))
fig.savefig('CGAN_CIFAR_SMOOTH.pdf')

## 4 WGAN

### 4.1 MNIST

In [ ]:
experiments = {
    'NCRIT5_CLIP0.99_LR5e-05'   : r'NCRIT=5  CLIP=0.99  $\eta$=5e-5 BN=No',   # 0
    'NCRIT5_CLIP0.99_LR1e-04'   : r'NCRIT=5  CLIP=0.99  $\eta$=1e-4 BN=No',   # 1
    'NCRIT5_CLIP0.99_LR5e-03'   : r'NCRIT=5  CLIP=0.99  $\eta$=5e-3 BN=No',   # 2
    'NCRIT5_CLIP0.001_LR1e-03'  : r'NCRIT=5  CLIP=0.001 $\eta$=1e-3 BN=No',   # 3
    'NCRIT5_CLIP0.01_LR1e-03'   : r'NCRIT=5  CLIP=0.01  $\eta$=1e-3 BN=No',   # 4
    'NCRIT5_CLIP0.1_LR1e-03'    : r'NCRIT=5  CLIP=0.10  $\eta$=1e-3 BN=No',   # 5
    'NCRIT5_CLIP0.5_LR1e-03'    : r'NCRIT=5  CLIP=0.50  $\eta$=1e-3 BN=No',   # 6
    'NCRIT5_CLIP0.99_LR1e-03'   : r'NCRIT=5  CLIP=0.99  $\eta$=1e-3 BN=No',   # 7
    'NCRIT5_CLIP2.0_LR1e-03'    : r'NCRIT=5  CLIP=2.00  $\eta$=1e-3 BN=No',   # 8
    'NCRIT1_CLIP0.99_LR1e-03'   : r'NCRIT=1  CLIP=0.99  $\eta$=1e-3 BN=No',   # 9
    'NCRIT20_CLIP0.99_LR1e-03'  : r'NCRIT=20 CLIP=0.99  $\eta$=1e-3 BN=No',   # 10
    'NCRIT5_CLIP0.99_LR1e-04_BN': r'NCRIT=5  CLIP=0.99  $\eta$=1e-4 BN=Yes',  # 11
    'NCRIT5_CLIP0.99_LR5e-03_BN': r'NCRIT=5  CLIP=0.99  $\eta$=5e-3 BN=Yes',  # 12
    'NCRIT5_CLIP0.1_LR1e-03_BN' : r'NCRIT=5  CLIP=0.10  $\eta$=1e-3 BN=Yes',  # 13
    'NCRIT5_CLIP0.5_LR1e-03_BN' : r'NCRIT=5  CLIP=0.50  $\eta$=1e-3 BN=Yes',  # 14
    'NCRIT5_CLIP0.99_LR1e-03_BN': r'NCRIT=5  CLIP=0.99  $\eta$=1e-3 BN=Yes',  # 15
    'NCRIT5_CLIP2.0_LR1e-03_BN' : r'NCRIT=5  CLIP=2.00  $\eta$=1e-3 BN=Yes',  # 16
    'NCRIT1_CLIP0.99_LR1e-03_BN': r'NCRIT=1  CLIP=0.99  $\eta$=1e-3 BN=Yes'   # 17
}
parent = os.path.join(wgan_folder, 'MNIST')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 6))
ax1.set_ylim([1.5, 7.5])
ax1.legend(loc='best', framealpha=0.8, fontsize='x-small', ncol=2)
ax2.legend(loc='best', framealpha=1, fontsize='x-small', ncol=2)
fig.savefig('WGAN_MNIST.pdf')

Highlight different critic steps

In [ ]:
lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
ids = {
    'ncrit=1' : [9, 17],
    'ncrit=5' : [*range(9), *range(11, 17)],
    'ncrit=20': [10]
}
[(lines1[i].set_color(colors[0]), lines2[i].set_color(colors[0])) for i in ids['ncrit=1']]
[(lines1[i].set_color(colors[1]), lines2[i].set_color(colors[1])) for i in ids['ncrit=5']]
[(lines1[i].set_color(colors[2]), lines2[i].set_color(colors[2])) for i in ids['ncrit=20']]
[
    axes.legend(
        [lines1[9], lines1[0], lines1[10]],
        ['Critic Steps = 1', 'Critic Steps = 5', 'Critic Steps = 20']
    ) for axes in [ax1, ax2]
]
fig.set_size_inches((12, 4))
plt.tight_layout(h_pad=1)
fig.savefig('WGAN_MNIST_NCRIT.pdf')

Highlight clip value

In [ ]:
lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
ids = {
    'clip=0.001' : [3],
    'clip=0.01' : [4],
    'clip=0.1': [5, 13],
    'clip=0.5': [6, 14],
    'clip=0.99': [*range(3), 7, *range(9, 13), 15, 17],
    'clip=2.0': [8, 16]
}
[(lines1[i].set_color(colors[0]), lines2[i].set_color(colors[0])) for i in ids['clip=0.001']]
[(lines1[i].set_color(colors[1]), lines2[i].set_color(colors[1])) for i in ids['clip=0.01' ]]
[(lines1[i].set_color(colors[2]), lines2[i].set_color(colors[2])) for i in ids['clip=0.1'  ]]
[(lines1[i].set_color(colors[3]), lines2[i].set_color(colors[3])) for i in ids['clip=0.5'  ]]
[(lines1[i].set_color(colors[4]), lines2[i].set_color(colors[4])) for i in ids['clip=0.99' ]]
[(lines1[i].set_color(colors[5]), lines2[i].set_color(colors[5])) for i in ids['clip=2.0'  ]]
[
    axes.legend(
        [lines1[3], lines1[4], lines1[5], lines1[6], lines1[7],  lines1[8]],
        ['Clip = 0.001', 'Clip = 0.01', 'Clip = 0.1', 'Clip = 0.5', 'Clip = 0.99', 'Clip = 2.0'],
        ncol=2
    ) for axes in [ax1, ax2]
]
fig.set_size_inches((12, 4))
plt.tight_layout(h_pad=1)
fig.savefig('WGAN_MNIST_CLIP.pdf')

Highlight use of Batch Normalization

In [ ]:
lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
ids = {
    'BN=No' : [*range(11)],
    'BN=Yes' : [*range(11, 18)],
}
[(lines1[i].set_color(colors[0]), lines2[i].set_color(colors[0])) for i in ids['BN=No']]
[(lines1[i].set_color(colors[1]), lines2[i].set_color(colors[1])) for i in ids['BN=Yes' ]]
[axes.legend([lines1[0], lines1[-1]], ['No Batch Normalization', 'Using Batch Normalization']) for axes in [ax1, ax2]]
fig.set_size_inches((12, 4))
plt.tight_layout(h_pad=1)
fig.savefig('WGAN_MNIST_BN.pdf')

In [ ]:
lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
ids = {
    '\eta=5e-5': [0],
    '\eta=1e-4': [1, 11],
    '\eta=1e-3': [*range(3, 11), *range(13, 18)],
    '\eta=5e-3': [2, 12],
}
[(lines1[i].set_color(colors[0]), lines2[i].set_color(colors[0])) for i in ids['\eta=5e-5']]
[(lines1[i].set_color(colors[1]), lines2[i].set_color(colors[1])) for i in ids['\eta=1e-4']]
[(lines1[i].set_color(colors[2]), lines2[i].set_color(colors[2])) for i in ids['\eta=1e-3']]
[(lines1[i].set_color(colors[3]), lines2[i].set_color(colors[3])) for i in ids['\eta=5e-3']]
[
    axes.legend(
        [lines1[0], lines1[1], lines1[3], lines1[2]],
        [r'$\eta$=5e-5', r'$\eta$=1e-4', r'$\eta$=1e-3', r'$\eta$=5e-3']
    ) for axes in [ax1, ax2]
]
fig.set_size_inches((12, 4))
plt.tight_layout(h_pad=1)
fig.savefig('WGAN_MNIST_learning_rate.pdf')

### 4.2 Fashion MNIST

In [ ]:
experiments = {
    'NCRIT5_CLIP0.01_LR5e-05' : r'NCRIT=5  CLIP=0.01  $\eta$=5e-5',  # 0
    'NCRIT5_CLIP0.5_LR5e-05'  : r'NCRIT=5  CLIP=0.50  $\eta$=5e-5',  # 1
    'NCRIT5_CLIP0.99_LR5e-05' : r'NCRIT=5  CLIP=0.99  $\eta$=5e-5',  # 2
    'NCRIT5_CLIP0.99_LR1e-03' : r'NCRIT=5  CLIP=0.99  $\eta$=1e-3',  # 3
    'NCRIT5_CLIP0.99_LR5e-03' : r'NCRIT=5  CLIP=0.99  $\eta$=5e-3',  # 4
    'NCRIT5_CLIP0.99_LR2e-04' : r'NCRIT=5  CLIP=0.99  $\eta$=2e-4',  # 5
    'NCRIT5_CLIP2.0_LR5e-05'  : r'NCRIT=5  CLIP=2.00  $\eta$=5e-5',  # 6
    'NCRIT10_CLIP0.99_LR1e-03': r'NCRIT=10 CLIP=0.99  $\eta$=1e-3'   # 7
}
parent = os.path.join(wgan_folder, 'Fashion')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd})
[axes.set_xlim([-1, 20.5]) for axes in [ax1, ax2]]
[axes.set_xticks([*range(0, 24, 4)]) for axes in [ax1, ax2]]
[axes.legend(loc='best', framealpha=1, fontsize='small', ncol=2) for axes in [ax1, ax2]]
fig.savefig('WGAN_Fashion.pdf')

In [ ]:
lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
ids = {
    '\eta=5e-5': [0, 1, 2, 6],
    '\eta=2e-4': [5],
    '\eta=1e-3': [3, 7],
    '\eta=5e-3': [4],
}
[(lines1[i].set_color(colors[0]), lines2[i].set_color(colors[0])) for i in ids['\eta=5e-5']]
[(lines1[i].set_color(colors[1]), lines2[i].set_color(colors[1])) for i in ids['\eta=2e-4']]
[(lines1[i].set_color(colors[2]), lines2[i].set_color(colors[2])) for i in ids['\eta=1e-3']]
[(lines1[i].set_color(colors[3]), lines2[i].set_color(colors[3])) for i in ids['\eta=5e-3']]
[
    axes.legend(
        [lines1[0], lines1[5], lines1[3], lines1[4]],
        [r'$\eta$=5e-5', r'$\eta$=2e-4', r'$\eta$=1e-3', r'$\eta$=5e-3']
    ) for axes in [ax1, ax2]
]
fig.set_size_inches((12, 4))
plt.tight_layout(h_pad=1)
fig.savefig('WGAN_Fashion_learning_rate.pdf')

## 5 WGAN-GP

### 5.1 MNIST

In [ ]:
experiments = {
    'NCRIT1_LAMBDA10' : r'NCRIT=1  $\lambda$=10',
    'NCRIT5_LAMBDA5'  : r'NCRIT=5  $\lambda$=5' ,
    'NCRIT5_LAMBDA10' : r'NCRIT=5  $\lambda$=10',
    'NCRIT5_LAMBDA20' : r'NCRIT=5  $\lambda$=20',
    'NCRIT10_LAMBDA10': r'NCRIT=10 $\lambda$=10'
}
parent = os.path.join(wgan_gp_folder, 'MNIST')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 4))
[axes.legend(loc='best', framealpha=1, fontsize='small') for axes in [ax1, ax2]]
fig.savefig('WGAN-GP_MNIST.pdf')

### 5.2 Fashion MNIST

In [ ]:
experiments = {
    'NCRIT1_LAMBDA5' : r'NCRIT=1 $\lambda$=5',
    'NCRIT5_LAMBDA5' : r'NCRIT=5 $\lambda$=5',
    'NCRIT5_LAMBDA10': r'NCRIT=5 $\lambda$=10',
    'NCRIT5_LAMBDA20': r'NCRIT=5 $\lambda$=20',
    'NCRIT10_LAMBDA5': r'NCRIT=10 $\lambda$=5'
}
parent = os.path.join(wgan_gp_folder, 'Fashion')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 4))
[axes.legend(loc='best', framealpha=1, fontsize='small') for axes in [ax1, ax2]]
fig.savefig('WGAN-GP_Fashion.pdf')

### 5.3 CIFAR

In [ ]:
experiments = {
    'NCRIT5_beta1[0.0]_LR2e-04_TRPCONV' : r'NCRIT=5 $\eta$=2e-4 $\beta_1$=0.0 UP=TrpConv',
    'NCRIT5_beta1[0.5]_LR2e-04_TRPCONV' : r'NCRIT=5 $\eta$=2e-4 $\beta_1$=0.5 UP=TrpConv',
    'NCRIT5_beta1[0.0]_LR2e-04_BILINEAR': r'NCRIT=5 $\eta$=2e-4 $\beta_1$=0.0 UP=Bilinear',
    'NCRIT5_beta1[0.5]_LR2e-04_BILINEAR': r'NCRIT=5 $\eta$=2e-4 $\beta_1$=0.5 UP=Bilinear',
    'NCRIT5_beta1[0.0]_LR2e-04_NEAREST' : r'NCRIT=5 $\eta$=2e-4 $\beta_1$=0.0 UP=Nearest',
    'NCRIT5_beta1[0.5]_LR2e-04_NEAREST' : r'NCRIT=5 $\eta$=2e-4 $\beta_1$=0.5 UP=Nearest',
    'NCRIT1_beta1[0.0]_LR1e-04_NEAREST' : r'NCRIT=1 $\eta$=1e-4 $\beta_1$=0.0 UP=Nearest'
}
parent = os.path.join(wgan_gp_folder, 'CIFAR')
cs, fcd = get_logs(parent, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 6))
[axes.legend(loc='best', framealpha=1, fontsize='x-small') for axes in [ax1, ax2]]
ax2.set_yticks([*range(0, 100, 10)])
ax2.set_ylim([0, 90])
fig.savefig('WGAN-GP_CIFAR.pdf')

## 6 Comparison

### 6.1 MNIST

In [ ]:
experiments = {
    os.path.join('DCGAN', 'MNIST', 'BS32_LDIM12_BILINEAR'): 'DCGAN 1',
    os.path.join('DCGAN', 'MNIST', 'BS32_LDIM50_BILINEAR'): 'DCGAN 2',
    os.path.join('DCGAN', 'MNIST', 'BS16_LDIM100_TRPCONV'): 'DCGAN 3',
    
    os.path.join('CGAN', 'MNIST', 'BS16'       ): 'CGAN 1',
    os.path.join('CGAN', 'MNIST', 'BS16_SMOOTH'): 'CGAN 2',
    os.path.join('CGAN', 'MNIST', 'BS32_SMOOTH'): 'CGAN 3',
    
    os.path.join('WGAN', 'MNIST', 'NCRIT1_CLIP0.99_LR1e-03'): 'WGAN 1',
    os.path.join('WGAN', 'MNIST', 'NCRIT5_CLIP0.99_LR5e-03'): 'WGAN 2',
    os.path.join('WGAN', 'MNIST', 'NCRIT5_CLIP0.99_LR1e-03'): 'WGAN 3',
    
    os.path.join('WGAN-GP', 'MNIST', 'NCRIT1_LAMBDA10'): 'WGAN-GP 1',
    os.path.join('WGAN-GP', 'MNIST', 'NCRIT5_LAMBDA5' ): 'WGAN-GP 2',
    os.path.join('WGAN-GP', 'MNIST', 'NCRIT5_LAMBDA10'): 'WGAN-GP 3'
}
cs, fcd = get_logs(top_folder, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 4))


lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
[line.set_color(colors[0]) for line in [*lines1[0:3], *lines2[0:3]]]
[line.set_color(colors[1]) for line in [*lines1[3:6], *lines2[3:6]]]
[line.set_color(colors[2]) for line in [*lines1[6:9], *lines2[6:9]]]
[line.set_color(colors[3]) for line in [*lines1[9:12], *lines2[9:12]]]
[
    axes.legend(
        [lines1[0], lines1[3], lines1[6], lines1[9]],
        ['DCGAN', 'CGAN', 'WGAN', 'WGAN-GP'],
        framealpha=1
    ) for axes in [ax1, ax2]
]
fig.savefig('Comparison_MNIST.pdf')

### 6.2 Fashion MNIST

In [ ]:
experiments = {
    os.path.join('DCGAN', 'Fashion', 'BS16_LDIM16_TRPCONV'): 'DCGAN 1',
    os.path.join('DCGAN', 'Fashion', 'BS16_LDIM32_TRPCONV'): 'DCGAN 2',
    os.path.join('DCGAN', 'Fashion', 'BS8_LDIM32_NEAREST'): 'DCGAN 3',
    
    os.path.join('CGAN', 'Fashion', 'BS16'       ): 'CGAN 1',
    os.path.join('CGAN', 'Fashion', 'BS32'       ): 'CGAN 2',
    os.path.join('CGAN', 'Fashion', 'BS16_SMOOTH'): 'CGAN 3',
    
    os.path.join('WGAN', 'Fashion', 'NCRIT5_CLIP0.99_LR1e-03' ): 'WGAN 1',
    os.path.join('WGAN', 'Fashion', 'NCRIT10_CLIP0.99_LR1e-03'): 'WGAN 2',
    os.path.join('WGAN', 'Fashion', 'NCRIT5_CLIP0.99_LR2e-04' ): 'WGAN 3',
    
    os.path.join('WGAN-GP', 'Fashion', 'NCRIT1_LAMBDA5' ): 'WGAN-GP 1',
    os.path.join('WGAN-GP', 'Fashion', 'NCRIT5_LAMBDA5' ): 'WGAN-GP 2',
    os.path.join('WGAN-GP', 'Fashion', 'NCRIT5_LAMBDA20'): 'WGAN-GP 3'
}
cs, fcd = get_logs(top_folder, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 4))


lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
[line.set_color(colors[0]) for line in [*lines1[0:3], *lines2[0:3]]]
[line.set_color(colors[1]) for line in [*lines1[3:6], *lines2[3:6]]]
[line.set_color(colors[2]) for line in [*lines1[6:9], *lines2[6:9]]]
[line.set_color(colors[3]) for line in [*lines1[9:12], *lines2[9:12]]]
[
    axes.legend(
        [lines1[0], lines1[3], lines1[6], lines1[9]],
        ['DCGAN', 'CGAN', 'WGAN', 'WGAN-GP'],
        framealpha=1
    ) for axes in [ax1, ax2]
]
fig.savefig('Comparison_Fashion.pdf')

### 6.3 CIFAR

In [ ]:
experiments = {
    os.path.join('DCGAN', 'CIFAR', 'TRPCONV'        ): 'DCGAN 1',
    os.path.join('DCGAN', 'CIFAR', 'TRPCONV_SMOOTH' ): 'DCGAN 2',
    os.path.join('DCGAN', 'CIFAR', 'BILINEAR_SMOOTH'): 'DCGAN 3',
    
    os.path.join('CGAN', 'CIFAR', 'beta1[0.0]_TRPCONV_SMOOTH'    ): 'CGAN 1',
    os.path.join('CGAN', 'CIFAR', 'beta1[0.0]_TRPCONV'           ): 'CGAN 2',
    os.path.join('CGAN', 'CIFAR', 'beta1[0.5]_BILINEAR_SMOOTH'): 'CGAN 3',
    
    os.path.join('WGAN-GP', 'CIFAR', 'NCRIT1_beta1[0.0]_LR1e-04_NEAREST' ): 'WGAN-GP 1',
    os.path.join('WGAN-GP', 'CIFAR', 'NCRIT5_beta1[0.0]_LR2e-04_TRPCONV' ): 'WGAN-GP 2',
    os.path.join('WGAN-GP', 'CIFAR', 'NCRIT5_beta1[0.0]_LR2e-04_BILINEAR'): 'WGAN-GP 3'
}
cs, fcd = get_logs(top_folder, experiments)
fig, (ax1, ax2) = plot_metrics({'CS': cs, 'FCD': fcd}, figsize=(12, 4))


lines1 = ax1.get_lines()
lines2 = ax2.get_lines()
[line.set_color(colors[0]) for line in [*lines1[0:3], *lines2[0:3]]]
[line.set_color(colors[1]) for line in [*lines1[3:6], *lines2[3:6]]]
[line.set_color(colors[3]) for line in [*lines1[6:9], *lines2[6:9]]]
[
    axes.legend(
        [lines1[0], lines1[3], lines1[6],],
        ['DCGAN', 'CGAN', 'WGAN-GP'],
        framealpha=1
    ) for axes in [ax1, ax2]
]
fig.savefig('Comparison_CIFAR.pdf')

## 7 Others

### 7.1 Showing labels of CIFAR

In [ ]:
best_image = os.path.join(cgan_folder, 'CIFAR', 'beta1[0.0]_TRPCONV_SMOOTH', 'epoch-25.png')
best_image = tf.keras.preprocessing.image.load_img(best_image)
best_image = tf.keras.preprocessing.image.img_to_array(best_image, dtype='uint8')
fig = plt.figure()
plt.imshow(best_image)

height = plt.ylim()[0] - plt.ylim()[1]
ticks = np.linspace(0, height, 21)
ticks = ticks[range(1, 21, 2)]
labels = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

plt.yticks(ticks, labels)
plt.xticks([])
plt.xlabel('')
plt.tight_layout(h_pad=0)
fig.savefig('CGAN_CIFAR_Labeled.pdf')